## Microsoft Entra ID の概要

Microsoft Entra ID（旧 Azure Active Directory）は、Microsoft のクラウドベースの ID およびアクセス管理サービスです。Microsoft 365、Azure、およびその他数千の SaaS アプリケーションの中央 ID プロバイダーとして機能します。

主な機能:
* **シングルサインオン（SSO）** - ユーザーは一度認証するだけで複数のアプリケーションにアクセス可能
* **多要素認証（MFA）** - 追加の検証方法によるセキュリティ強化
* **条件付きアクセス** - ユーザー、デバイス、場所、リスクに基づくポリシーベースのアクセス制御
* **アプリケーション統合** - OAuth 2.0、OpenID Connect、SAML などの最新の認証プロトコルをサポート

## 学習目標
Microsoft Entra ID は、AgentCore Identity の ID プロバイダーとして使用でき、ユーザーを認証し、ユーザーに代わって保護されたリソースにアクセスするようエージェントを認可させることができます。

<img src="images/entra-notebook-overview.png" width="75%">

## 前提条件

このチュートリアルを実行するには以下が必要です：

* Python 3.10+
* AWS 認証情報
* Strands Agents
* Docker、Finch、または Podman がインストールされていること
* AWS リージョンを「us-west-2」または Bedrock AgentCore をサポートする任意のリージョンに設定。サポートされているリージョンについては https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/agentcore-regions.html を参照してください。

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## ステップ 1: Entra ID テナントのセットアップ

## 認証コードフロー
OAuth 2.0 認証コードフローは、Web アプリケーションがユーザーを安全に認証し、アクセストークンを取得するための推奨アプローチです。このフローには以下が含まれます：
1. ユーザーを認証のために Entra ID にリダイレクト
2. ログイン成功後に認証コードを受信
3. コードをアクセストークンとリフレッシュトークンに交換
4. トークンを使用して保護されたリソースにアクセス

この統合パターンにより、AgentCore はアプリケーションの安全で標準ベースの認証を維持しながら、Entra ID の堅牢な ID 管理機能を活用できます。

Entra ID テナントは、組織を表す Microsoft Entra ID の専用インスタンスです。Microsoft のクラウド内にある組織の独立したディレクトリと考えてください。

主な特徴:
* **一意の ID** - 各テナントは一意のドメインを持ちます（例：yourcompany.onmicrosoft.com）
* **分離された境界** - あるテナント内のユーザー、グループ、アプリケーションは他のテナントから分離されています
* **管理制御** - テナント管理者がユーザー、セキュリティポリシー、アプリケーション登録を管理します
* **マルチドメインサポート** - デフォルトの .onmicrosoft.com ドメインとカスタムドメインの両方を含めることができます

実際の運用:

OAuth 統合のために Entra ID にアプリケーションを登録する場合、特定のテナント内に登録することになります。そのテナントのユーザーは、組織の資格情報を使用してアプリケーションに認証できます。

AgentCore 統合に必要なもの:
* **テナント ID** - Entra ID インスタンスの一意の識別子
* **アプリケーション登録** - テナント内に登録されたアプリ
* **適切な権限** - アプリケーションに設定されたアクセス権

このテナントベースのモデルにより、認証と認可が組織のセキュリティ境界内に維持されます。

テナントの作成手順については https://learn.microsoft.com/en-us/entra/fundamentals/create-new-tenant を参照してください

注意:
1. Microsoft Entra ID は AWS のサービスではありません。コストに関連する詳細については、Microsoft Entra ID のドキュメントを参照してください。
2. 以下のステップで使用されているスクリーンショットは変更される可能性があります。Entra ID アプリケーションのセットアップに関する最新のガイダンスについては、Microsoft Entra ID のドキュメントを参照することをお勧めします。

## ステップ 2: アプリケーションのセットアップ

1. https://portal.azure.com にアクセスし、画面上部の検索バーで「Entra ID」を検索します
<img src="images/entraid.jpg" width="75%">

2. `管理` &rarr; `アプリの登録` に移動します
<img src="images/app.registration.png" width="75%">


3. `新規登録` をクリックし、詳細を入力します。必ずマルチテナントオプションを選択してください
<img src="images/app.registration.form.png" width="75%">


4. クライアントシークレットを作成します。AgentCore Identity で使用するために clientId とクライアントシークレットをコピーします。
<img src="images/gather.client.info.png" width="75%">


5. OAuth 用のスコープを作成します。API の公開 &rarr; `スコープの追加` に移動します。完全なスコープをコピーして保存します。
<img src="images/expose.api.png" width="75%">


6. OneNote へのアクセスを許可する API アクセス許可を追加します。
<img src="images/onenote.api.perm.png" width="75%"/>


7. 発行するトークンを選択します。
<img src="images/access.token.issue.png" width="75%"/>

## ステップ 2 - Bedrock AgentCore Identity プロバイダーの作成

ステップ 1 で作成したテナントとアプリケーションの詳細を使用して、以下の環境変数を更新します。

In [ ]:
import os

# REPLACE WITH YOUR client_id
os.environ["client_id"] = "REPLACE_ME"

# REPLACE WITH YOUR secret
os.environ["secret"] = "REPLACE_ME"

# REPLACE WITH YOUR tenant_id
os.environ["tenant_id"] = "REPLACE_ME"

##########

# REPLACE WITH YOUR scopes, if needed
os.environ["scopes"] = (
    "openid profile https://graph.microsoft.com/Notes.ReadWrite.All https://graph.microsoft.com/Notes.Create"
)

# REPLACE WITH YOUR audience, if needed
os.environ["audience"] = "https://graph.microsoft.com"

Amazon Bedrock AgentCore Identity は、Inbound 認証と Outbound 認証の両方に対して、マネージド OAuth 2.0 対応プロバイダーを提供します。

エージェントで使用する ID プロバイダーを作成します。プロバイダーは、さまざまな OAuth 2.0 実装、API 認証スキーム、トークン形式の複雑さを抽象化し、基盤となるプロトコルのバリエーションやエッジケースを処理しながら、エージェントに統一されたインターフェースを提供します。

In [ ]:
from bedrock_agentcore.services.identity import IdentityClient
from boto3.session import Session


boto_session = Session()
region = boto_session.region_name

if not region:
    import warnings

    warnings.warn(
        "There is no configured Region in the AWS session. Defaulting to us-east-1"
    )
    region = "us-east-1"

identity_client = IdentityClient(region=region)

ms_provider = identity_client.create_oauth2_credential_provider(
    req={
        "name": "microsoft_entra_oauth_provider",
        "credentialProviderVendor": "MicrosoftOauth2",
        "oauth2ProviderConfigInput": {
            "microsoftOauth2ProviderConfig": {
                "clientId": os.environ["client_id"],
                "clientSecret": os.environ["secret"],
                "tenantId": os.environ["tenant_id"],
            }
        },
    }
)
print(f"Microsoft Credential Provider: {ms_provider}")
print()
print(f"Callback URL: {ms_provider['callbackUrl']}")

## ステップ 2.5: Credential Provider からのコールバック URL で Microsoft Entra を更新

<img src="images/redirect.uri.png" width="75%">

## ステップ 3: ローカルでの検証

AgentCore Identity により、開発者は設定された OAuth 2.0 認証情報プロバイダーに基づいて、ユーザー委任アクセスまたはマシン間認証のいずれかの OAuth トークンを取得できます。

このサービスは、ユーザーまたはアプリケーションとダウンストリームの認可サーバー間の認証プロセスを調整し、結果のトークンを取得して保存します。トークンが AgentCore Identity ボールトで利用可能になると、認可されたエージェントはそれを取得し、リソースサーバーへの呼び出しを認可するために使用できます。

##### 以下のコードでは、ユーザー委任フローに Entra ID を使用しています。

In [ ]:
from bedrock_agentcore.identity.auth import requires_access_token
from oauth2_callback_server import get_oauth2_callback_url


@requires_access_token(
    provider_name="microsoft_entra_oauth_provider",
    auth_flow="USER_FEDERATION",
    scopes=os.environ["scopes"].split(" "),
    on_auth_url=lambda x: print(
        "\nPlease copy and paste this URL in your browser:\n" + x
    ),
    force_authentication=True,
    callback_url=get_oauth2_callback_url(),
)
def need_access_token(*, access_token: str):
    return access_token

##### `need_access_token(access_token="")` は、Entra ID に認証してアプリケーションが使用する認可トークンを取得するための URL を表示します。認証して同意を共有すると、認証コードが利用可能になります。

<img src="images/authenticate.and.authorize.png" width="75%">

In [ ]:
import sys
import subprocess

from oauth2_callback_server import wait_for_oauth2_server_to_be_ready


oauth2_callback_server_cmd = [
    sys.executable,
    "oauth2_callback_server.py",
    "--region",
    region,
]
oauth2_callback_server_process = subprocess.Popen(oauth2_callback_server_cmd)

id_token = ""
try:
    successfully_started_oauth2_server = wait_for_oauth2_server_to_be_ready()
    if not successfully_started_oauth2_server:
        print(
            "Failed to start OAuth2 callback server to handle session binding "
            "(https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/oauth2-authorization-url-session-binding.html)"
        )
    else:
        id_token = need_access_token(access_token="")
        print(f"Bearer Token Received: {id_token[:10]}...")
finally:
    oauth2_callback_server_process.terminate()

##### トークンをデコードしてローカルで検証できます。

In [ ]:
import json

from jwt import JWT


jwt = JWT()

# Decode the token (without verification for inspection purposes only)
# For production, always verify the token's signature and claims
decoded_token = jwt.decode(id_token, do_verify=False)

print(f"Decoded Bearer Token (for inspection): \n{json.dumps(decoded_token, indent=4)}")

##### Entra ID からのデコードされたトークンは以下のようになるはずです。
<img src="images/decoded-token.png" width="75%">

## ステップ 4 - すべてを AgentCore Runtime エージェントとして統合

### OneNote 統合エージェント

このコードは、自然言語コマンドを通じてユーザーが Microsoft OneNote ノートブックを作成・管理するのを支援する AI エージェントを作成します。このエージェントは EntraID 認証を使用して OneNote API にアクセスし、3 つの主要な機能を提供します：

1. ノートブックの作成 - 新しい OneNote ノートブックを作成（`create_notebook` ツール）
2. セクションの作成 - 既存のノートブックにセクションを追加（`create_notebook_section` ツール）
3. コンテンツの追加 - ノートブックセクションにコンテンツ付きのページを作成（`add_content_to_notebook_section` ツール）

エージェントは OAuth2 認証を自動的に処理し、必要に応じてユーザーに認可を求め、会議のメモやその他のコンテンツを構造化された OneNote ノートブックに整理するリクエストを処理します。

In [ ]:
%%writefile strands_entraid_onenote.py
import os
import json
import asyncio
import requests

from strands import Agent
from strands import tool
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from bedrock_agentcore.identity.auth import requires_access_token
from strands.models.bedrock import BedrockModel
from oauth2_callback_server import get_oauth2_callback_url

os.environ["STRANDS_OTEL_ENABLE_CONSOLE_EXPORT"] = "true"
os.environ["OTEL_PYTHON_EXCLUDED_URLS"] = "/ping,/invocations"

entra_access_token = None  # Global variable to store the access token
tool_name = None

@tool
def create_notebook(name: str) -> str:
    """
    Create a new Microsoft OneNote notebook for the user. Needed before you can create a section or add content.
    
    Args:
        name (str): The display name for the new notebook
        
    Returns:
        str: The ID of the created notebook
    """
    global entra_access_token
    global tool_name 
    tool_name = "create_notebook"
    # Check if we already have a token
    if not entra_access_token:
        return json.dumps({"auth_required": True, "message": f"Entra ID authentication is required for {tool_name}. Please wait while we set up the authorization.", "events": []})

    headers = {
        'Authorization': f'Bearer {entra_access_token}',
        'Content-Type': 'application/json'
    }
    # Create new notebook
    notebook_data = {'displayName': name}
    notebook = requests.post(
        'https://graph.microsoft.com/v1.0/me/onenote/notebooks', 
        headers=headers, 
        json=notebook_data
    )
    notebook.raise_for_status()
    return json.dumps({"notebook_id": notebook.json()['id']})

@tool
def create_notebook_section(notebook_id: str, section_name: str) -> str:
    """
    Create a new section in an existing OneNote notebook. Section is created for a specific notebook. 
    
    Args:
        notebook_id (str): The ID of the OneNote notebook to create the section in
        section_name (str): The display name for the new section
        
    Returns:
        str: The ID of the created section
    """
    global entra_access_token
    global tool_name 
    tool_name = "create_notebook_section"
    # Check if we already have a token
    if not entra_access_token:
        return json.dumps({"auth_required": True, "message": f"Entra ID authentication is required for {tool_name}. Please wait while we set up the authorization.", "events": []})

    headers = {
        'Authorization': f'Bearer {entra_access_token}',
        'Content-Type': 'application/json'
    }
    # Create new section
    section_data = {'displayName': section_name}
    section = requests.post(
        f'https://graph.microsoft.com/v1.0/me/onenote/notebooks/{notebook_id}/sections',
        headers=headers, 
        json=section_data
    )
    section.raise_for_status()
    
    section_id = section.json()['id']
    return json.dumps({"section_id": section_id})

@tool
def add_content_to_notebook_section(section_id: str, page_content) -> str:
    """
    Add content to a OneNote notebook section by creating a new page.
    
    Args:
        section_id (str): The ID of the OneNote section to add content to
        page_content: The HTML content to add as a new page
        
    Returns:
        str: URL to the created notebook page
    """
    global entra_access_token
    global tool_name 
    tool_name = "add_content_to_notebook_section"
    
    # Check if we already have a token
    if not entra_access_token:
        return json.dumps({"auth_required": True, "message": f"Entra ID authentication is required for {tool_name}. Please wait while we set up the authorization.", "events": []})

    headers = {
        'Authorization': f'Bearer {entra_access_token}',
        'Content-Type': 'text/html'
    }
    page = requests.post(
        f'https://graph.microsoft.com/v1.0/me/onenote/sections/{section_id}/pages',
        headers=headers, 
        data=page_content
    )
    page.raise_for_status()
    url = json.loads(page.text)["links"]["oneNoteWebUrl"]["href"]
    return json.dumps({"oneNoteWebUrl": url})
    
    
# Initialize the agent with tools
model = BedrockModel(model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0")
system_prompt = """You are an Agent who helps user put in their meeting into OneNote notebooks. 
    Identify the notebook name, section name and content based on what the user has provided. 
    Return notebook URL once created."""
agent = Agent(model=model, system_prompt=system_prompt, tools=[create_notebook, create_notebook_section, add_content_to_notebook_section])

# Initialize app and streaming queue
app = BedrockAgentCoreApp()

class StreamingQueue:
    def __init__(self):
        self.finished = False
        self.queue = asyncio.Queue()
        
    async def put(self, item):
        await self.queue.put(item)

    async def finish(self):
        self.finished = True
        await self.queue.put(None)

    async def stream(self):
        while True:
            item = await self.queue.get()
            if item is None and self.finished:
                break
            yield item

queue = StreamingQueue()

async def on_auth_url(url: str):
    print(f"Authorization url: {url}")
    await queue.put(f"Authorization url: {url}")


async def agent_task(user_message: str):
    global tool_name
    try:
        await queue.put("Begin agent execution")
        
        # Call the agent first to see if it needs authentication
        response = agent(user_message)
        
        # Extract text content from the response structure
        response_text = ""
        if isinstance(response.message, dict):
            content = response.message.get('content', [])
            if isinstance(content, list):
                for item in content:
                    if isinstance(item, dict) and 'text' in item:
                        response_text += item['text']
        else:
            response_text = str(response.message)
        
        # Check if the response indicates authentication is required
        # Look for various keywords that indicate authentication issues
        auth_keywords = [
            "authentication", "authorize", "authorization", "auth", 
            "sign in", "login", "access", "permission", "credential",
            "need authentication", "requires authentication"
        ]
        needs_auth = any(keyword.lower() in response_text.lower() for keyword in auth_keywords)
       
        if needs_auth:
            await queue.put(f"Authentication required for {tool_name} access. Starting authorization flow...")
            
            # Trigger the 3LO authentication flow
            try:
                global entra_access_token
                entra_access_token = await need_token_3LO_async(access_token=None)
                await queue.put(f"Authentication successful! Retrying {tool_name}...")
                
                # Retry the agent call now that we have authentication
                response = agent(user_message)
            except Exception as auth_error:
                print(f"auth_error: ", repr(auth_error))
                await queue.put(f"Authentication failed: {repr(auth_error)}")
        
        await queue.put(response.message)
        await queue.put("End agent execution")
    except Exception as e:
        await queue.put(f"Error: {repr(e)}")
    finally:
        await queue.finish()

@requires_access_token(
    provider_name="microsoft_entra_oauth_provider",
    scopes=os.environ["scopes"].split(' '),
    auth_flow='USER_FEDERATION',
    on_auth_url=on_auth_url,
    force_authentication=True,
    callback_url=get_oauth2_callback_url(),
)
async def need_token_3LO_async(*, access_token: str):
    global entra_access_token
    entra_access_token = access_token  # Update the global access token
    print("Got access token....", access_token)
    return access_token


@app.entrypoint
async def agent_invocation(payload):
    user_message = payload.get("prompt", "No prompt found in input, please guide customer to create a json payload with prompt key")
    
    # Create and start the agent task
    task = asyncio.create_task(agent_task(user_message))

    # Return the stream, but ensure the task runs concurrently
    async def stream_with_task():
        # Stream results as they come
        async for item in queue.stream():
            yield item
        
        # Ensure the task completes
        await task
    
    return stream_with_task()
    
if __name__ == "__main__":
    app.run()


##### AgentCore Runtime を設定する

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="strands_entraid_onenote.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="strands_entraid_onenote_3lo",
)

print(f"Agent Configure Response: {response}")

##### エージェントを起動する。起動すると、エージェントはアプリケーションで使用できるようになります

In [ ]:
launch_response = agentcore_runtime.launch(
    local_build=True,
    auto_update_on_conflict=True,
    env_vars={
        "scopes": os.environ["scopes"],
    },
)

print(f"Launch Response: {launch_response}")

#### 以下のコードセルで URL が表示されます。取得した URL（下の画像ではなく）をコピーして、ブラウザウィンドウで認証を行います。
<img src="images/url.presented.png" width="75%">

#### 認証を求められます。認証を完了してください。
<img src="images/authenticate.and.authorize.png" width="75%">

#### 「Bedrock Agents」という名前のノートブックがすでに存在する場合は、必ず削除してください。

In [ ]:
import sys
import uuid
import subprocess

from typing import Final
from oauth2_callback_server import (
    store_user_id_in_oauth2_callback_server,
    wait_for_oauth2_server_to_be_ready,
)


prompt = """
Put these notes into onenote notebook named "Bedrock Agents".

Amazon Bedrock AgentCore enables you to deploy and operate 
highly capable AI agents securely, at scale. It offers 
infrastructure purpose-built for dynamic agent workloads, 
powerful tools to enhance agents, and essential controls for 
real-world deployment. AgentCore services can be used 
 together or independently and work with any framework including 
CrewAI, LangGraph, LlamaIndex, and Strands Agents, as well as 
any foundation model in or outside of Amazon Bedrock, giving you 
ultimate flexibility. AgentCore eliminates the undifferentiated 
heavy lifting of building specialized agent infrastructure, so 
you can accelerate agents to production. Provide link to 
the created OneNote Notebook and provide error message from the API 
in case of failure.
"""
session_id = str(uuid.uuid1())
user_id: Final[str] = "user"

oauth2_callback_server_cmd = [
    sys.executable,
    "oauth2_callback_server.py",
    "--region",
    region,
]
oauth2_callback_server_process = subprocess.Popen(oauth2_callback_server_cmd)

try:
    successfully_started_oauth2_server = wait_for_oauth2_server_to_be_ready()
    if not successfully_started_oauth2_server:
        print(
            "Failed to start OAuth2 callback server to handle session binding "
            "(https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/oauth2-authorization-url-session-binding.html)"
        )
    else:
        store_user_id_in_oauth2_callback_server(user_id)
        st = agentcore_runtime.invoke(
            payload={"prompt": prompt}, session_id=session_id, user_id=user_id
        )
finally:
    oauth2_callback_server_process.terminate()

## ステップ 4 - 作成された OneNote ノートブックの確認

- 上記のエージェント呼び出し関数は、ノートブック、その中にセクション、およびセクションにコンテンツを追加します。
- ユーザーは https://sharepoint.com/ にログインして作成されたノートブックにアクセスできます

<img src="images/sharepoint.com.png" width="75%"/>
エージェント呼び出しのレスポンスで提供されたリンクを使用して、作成されたノートブックにアクセスします。
<img src="images/notebook.content.png" width="75%"/>
<img src="images/list.notebooks.png" width="75%"/>

## まとめとクリーンアップ

このノートブックでは以下のことを学びました：
- OAuth 認証コードフローを提供するための Entra ID API とアプリケーションのセットアップ
- AgentCore Runtime を作成し、ユーザーに代わって OneNote ノートブックを作成するツールを持つエージェントをデプロイ

#### 作成されたリソース

In [ ]:
print(f"Runtime Agent: {launch_response.agent_id}")

#### AgentCore Runtime を削除

In [ ]:
import os
import boto3

agentcore_control_client = boto3.client("bedrock-agentcore-control", region_name=region)

delete_agent_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_response.agent_id
)
print(delete_agent_response)
print()

try:
    os.remove(".bedrock_agentcore.yaml")
    print("Successfully deleted local Runtime Agent config")
except Exception as e:
    print(f"Failed to delete local Agent config: {repr(e)}")

#### OAuth2 credential provider を削除

In [ ]:
delete_credential_provider = agentcore_control_client.delete_oauth2_credential_provider(
    name=ms_provider["name"]
)
print(delete_credential_provider)